# Day 7

## Day 7.1

## Approach 1: Create a single expression by recursive substitution

In [20]:
binary_command = {'NOT': '~', 'AND': '&', 'OR': '|', 'LSHIFT': '<<', 'RSHIFT': '>>'}
operators = binary_command.values()

In [21]:
import csv

def translate(l):
    return [binary_command[a] if a in binary_command else a for a in l]

def display(input_file):

    """produce a dict mapping variables to expressions"""
    
    commands = []
    with open(input_file, 'rt') as f_input:
        csv_reader = csv.reader(f_input, delimiter=' ')
        for line in csv_reader:
            commands.append((line[-1], ' '.join(list(translate(line[:-2])))))
    return dict(commands)

In [32]:
import re

def extract_variables(expr):
    varbls = []
    regex_pattern = '\s|\\)|\\('
    l = re.split(regex_pattern, expr)
    for a in l:
        if (a not in operators) and (not a.isnumeric()) and (a != ''):
            varbls.append(a)
    return set(varbls)


def create_instance(wire):
    exec_python = commands[wire]
    pending = extract_variables(commands[wire])
    count = 0
    while pending and (count < 200):
        s = pending.pop()
        expr = commands[s]
        exec_python = re.sub('({0})'.format(s), '( {0} )'.format(expr), exec_python)
        pending = pending.union(extract_variables(exec_python))
        count += 1
    return wire + ' = ' + exec_python

def evaluate(var):
    instance = create_instance(var)
    exec(instance)
    return np.uint16(locals()[var])

### Test

In [40]:
commands = display('inputs/input7.test.txt')

In [41]:
def test():
    assert(evaluate('d') == 72)
    assert(evaluate('e') == 507)
    assert(evaluate('f') == 492)
    assert(evaluate('g') == 114)
    assert(evaluate('h') == 65412)
    assert(evaluate('i') == 65079)
    assert(evaluate('x') == 123)
    assert(evaluate('y') == 456)

In [42]:
test()

This approach seems correct, but it creates huge expressions along the way that become harder and harder to parse. Thus the time to a final expression that wraps up all the computations is very long. Two ideas to explore: i) concurrent evaluation of expressions; ii) define lazy variables/functions that collect all the dependencies of the circuit and start firing upon request.

## Approach 2: Keep evaluating the unknown variables from the known ones.

The solution provided hereto owes credit to this source: https://www.reddit.com/r/adventofcode/comments/5id6w0/2015_day_7_part_1_python_wrong_answer/

In [37]:
import numpy as np

def RSHIFT(a, b):
    result = np.uint16(a) >> int(b)
    return int(result)

def LSHIFT(a, b):
    result = np.uint16(a) << int(b)
    return int(result)

def OR(a, b):
    result = np.uint16(a) | np.uint16(b)
    return int(result)

def AND(a, b):
    result = np.uint16(a) & np.uint16(b)
    return int(result)

def NOT(a):
    result = ~ np.uint16(a)
    return int(result)

In [59]:
import csv
def display(input_file):

    """produce a dict mapping variables to expressions"""
    
    commands = []
    with open(input_file, 'rt') as f_input:
        csv_reader = csv.reader(f_input, delimiter=' ')
        for line in csv_reader:
            commands.append((line[-1], line[:-2]))
    return dict(commands)

In [105]:
def evaluate(wire):
    known = {}
    while wire not in known:
        if wire in known:
            break
        for k, v in commands.items():
            if (len(v) == 1) and (v[0].isnumeric()) and (k not in known):
                known[k] = int(v[0])
            elif (len(v) == 1) and (v[0] in known) and (k not in known):
                known[k] = known[v[0]]
            elif ('AND' in v) and (v[0] in known) and (v[2] in known):
                known[k] = AND(known[v[0]], known[v[2]])
            elif ('AND' in v) and (v[0].isnumeric()) and (v[2] in known):
                known[k] = AND(int(v[0]), known[v[2]])
            elif ('AND' in v) and (v[0] in known) and (v[2].isnumeric()):
                known[k] = AND(known[v[0]], int(v[2]))
            elif ('OR' in v) and (v[0] in known) and (v[2] in known):
                known[k] = OR(known[v[0]], known[v[2]])
            elif ('OR' in v) and (v[0].isnumeric()) and (v[2] in known):
                known[k] = OR(int(v[0]), known[v[2]])
            elif ('OR' in v) and (v[0] in known) and (v[2].isnumeric()):
                known[k] = OR(known[v[0]], int(v[2]))
            elif ('LSHIFT' in v) and (v[0] in known):
                known[k] = LSHIFT(known[v[0]], v[2])
            elif ('RSHIFT' in v) and (v[0] in known):
                known[k] = RSHIFT(known[v[0]], v[2])
            elif ('NOT' in v) and (v[1] in known):
                known[k] = NOT(known[v[1]])
    return known[wire]

### Test 0

In [107]:
commands = display('inputs/input7.test1.txt')
commands

{'a': ['b'], 'b': ['1', 'OR', 'x'], 'x': ['12']}

In [108]:
evaluate('a')

13

### Test 1

In [109]:
commands = display('inputs/input7.test2.txt')
commands

{'d': ['x', 'AND', 'y'],
 'e': ['x', 'OR', 'y'],
 'f': ['x', 'LSHIFT', '2'],
 'g': ['y', 'RSHIFT', '2'],
 'h': ['NOT', 'x'],
 'i': ['NOT', 'y'],
 'x': ['123'],
 'y': ['456']}

In [110]:
test()

### Solution

In [111]:
commands = display('inputs/input7.txt')
evaluate('a')

16076

## Approach 3: Lazy Variable Wrappers

In [113]:
# TODO

## Day 7.2

In [114]:
commands = display('inputs/input7.txt')
commands['b'] = ['16076']
evaluate('a')

2797